In [1]:
# 读取fileList
import numpy as np

fileList = np.load("/home/wngys/lab/DeepFold/protein_infor/entPathArray.npy", allow_pickle=True).tolist()

In [ ]:
# # GitHub 原代码 生成距离矩阵
# import sys
# import os
# import numpy
# import scipy
# import scipy.spatial

# from Bio.PDB import PDBParser


# def get_distance_matrix(pdb_path):
#     parser = PDBParser()
#     structure = parser.get_structure('structure', pdb_path).get_list()[0]
#     residue_positions = get_residue_positions(structure)
#     pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
#     pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
#     return pdb_dist_mat


# def get_residue_ids(structure):
#     ids = [r.get_id()[1] for r in structure.get_residues()]
#     return ids


# def get_residue_positions(structure):
#     residue_ids = get_residue_ids(structure)
#     positions = numpy.ones((residue_ids[-1] - residue_ids[0] + 1, 3)) * float('inf')
#     for residue in structure.get_residues():
#         atoms = residue.get_atoms()
#         for a in atoms:
#             if a.get_name() == 'CA':
#                 positions[residue.get_id()[1] - residue_ids[0]] = a.get_coord()

#     return positions

In [3]:
# 修改的 GitHub 原代码 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

from Bio.PDB import PDBParser


def get_distance_matrix_modified(pdb_path):
    parser = PDBParser()
    structure = parser.get_structure('structure', pdb_path).get_list()[0]
    residue_positions = get_residue_positions(structure)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(residue_positions, 'euclidean'))
    pdb_dist_mat[numpy.isnan(pdb_dist_mat)] = float('inf')
    return pdb_dist_mat


def get_residue_ids(structure):
    ids = [r.get_id()[1] for r in structure.get_residues()]
    # print(ids)
    # print(len(ids))
    return ids


def get_residue_positions(structure):
    residue_ids = get_residue_ids(structure)
    positions = numpy.ones((len(residue_ids), 3)) * float('inf')
    i = 0
    for residue in structure.get_residues():
        atoms = residue.get_atoms()
        for a in atoms:
            if a.get_name() == 'CA':
                positions[i] = a.get_coord()
                i = i + 1
    return positions

In [5]:
# # 测试改进后的获取距离矩阵的函数
# import warnings
# warnings.filterwarnings('ignore')

# pdb_path = "/home/wngys/scop/scope-2.07-40/cm/d3cmco1.ent"
# parser = PDBParser()
# structure = parser.get_structure('structure', pdb_path).get_list()[0]

# residue_list = get_residue_ids(structure)
# residue_positions = get_residue_positions(structure)

In [2]:
# 自己解析 ent文件 生成距离矩阵
import sys
import os
import numpy
import scipy
import scipy.spatial

def get_distance_matrix_mine(pdb_path):
    CA_positions = []
    residue_set = set()
    with open(pdb_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line[:4] != "ATOM":
                if line[:6] != "HETATM":
                    continue
            if line[13:15] == "CA":
                residue_id = line[22:27].strip()
                if residue_id not in residue_set:
                    position = [float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())]
                    CA_positions.append(position)
                    residue_set.add(residue_id)
    pdb_dist_mat = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(CA_positions, 'euclidean'))
    return pdb_dist_mat

In [3]:
# # 测试自己的获取距离矩阵的函数
# pdb_path = "/home/wngys/scop/scope-2.07-40/cm/d3cmco1.ent"
# print(get_distance_matrix_mine(pdb_path).shape)

(171, 171)


In [4]:
# 生成14000多个距离矩阵并保存
import warnings
warnings.filterwarnings('ignore')
import numpy as np

errorList = []
errorFile = []

# save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_inf/"
save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_mine/"
# save_dir = "/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_modified/"

i = 0

for file_n in fileList:
    i = i + 1
    if i % 1000 == 0:
        print(i)
    
    ID = file_n.split("/")[-1].split(".")[0]
    
    try:
        # M = get_distance_matrix(file_n)
        M = get_distance_matrix_mine(file_n)
        # M = get_distance_matrix_modified(file_n)
    except:
        errorList.append(ID)
        errorFile.append(file_n)
        print("error: " + ID)
    else:
        np.save(save_dir+ID+".npy", M)
        # print("done: " + ID)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [5]:
print(len(errorList))
print(len(errorFile))

0
0


In [ ]:
# get_distance_matrix_mine、get_distance_matrix_modified未产生报错的文件
# np.save("/home/wngys/lab/DeepFold/protein_infor/errorIDArrayMine.npy", errorList)
# np.save("/home/wngys/lab/DeepFold/protein_infor/errorEntPathArrayMine.npy", errorFile)

In [2]:
errorList = ['d1nria_', 'd3msza1', 'd1ngvb_', 'd1bd7a_', 'd4ax1b_', 'd1nova_']
errorFile = ['/home/wngys/scop/scope-2.07-40/nr/d1nria_.ent', 
'/home/wngys/scop/scope-2.07-40/ms/d3msza1.ent', 
'/home/wngys/scop/scope-2.07-40/ng/d1ngvb_.ent', 
'/home/wngys/scop/scope-2.07-40/bd/d1bd7a_.ent', 
'/home/wngys/scop/scope-2.07-40/ax/d4ax1b_.ent', 
'/home/wngys/scop/scope-2.07-40/no/d1nova_.ent']

np.save("/home/wngys/lab/DeepFold/protein_infor/errorIDArrayInf.npy", errorList)
np.save("/home/wngys/lab/DeepFold/protein_infor/errorEntPathArrayInf.npy", errorFile)

In [6]:
# 测试：加载举例矩阵
M_2 = np.load("/home/wngys/lab/DeepFold/distance_matrix/distance_matrix_mine/d3cmco1.npy", allow_pickle=True)
print(M_2.shape)
print(M_2)

(171, 171)
[[ 0.          3.79202242  6.35042054 ... 13.2737778  12.00888013
  11.58735557]
 [ 3.79202242  0.          3.79668843 ... 11.7336577  10.18102657
   8.93318862]
 [ 6.35042054  3.79668843  0.         ... 14.86487524 13.03954696
  10.88075328]
 ...
 [13.2737778  11.7336577  14.86487524 ...  0.          3.8253341
   7.11461679]
 [12.00888013 10.18102657 13.03954696 ...  3.8253341   0.
   3.80227471]
 [11.58735557  8.93318862 10.88075328 ...  7.11461679  3.80227471
   0.        ]]
